In [1]:
import SimpleITK as sitk
import os
import glob
from pathlib import Path
import json
import dicom2nifti
from pprint import pprint

In [22]:
basedir = "/red/ruogu.fang/cox.j/SwinValidation/Pretrain/"
outbasedir = "/red/ruogu.fang/yyang/data/LUNA16"
folders = glob.glob(os.path.join(basedir, "subset_*"))
folders.append(os.path.join(basedir,"seg-lungs-LUNA16"))
for folder in folders:
    print(folder)

/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_2
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_9
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_6
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_4
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_5
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_8
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_0
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_1
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_3
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/subset_7
/red/ruogu.fang/cox.j/SwinValidation/Pretrain/seg-lungs-LUNA16


In [ ]:
def convert_LUNA16(basedir, outbasedir):
    folders = glob.glob(os.path.join(basedir, "subset_*"))
    folders.append(os.path.join(basedir,"seg-lungs-LUNA16"))
    for folder in folders:
        outfolder = os.path.join(outbasedir, os.path.basename(folder))
        if not os.path.exists(outfolder): os.makedirs(outfolder)
        print(outfolder)
        for filepath in glob.glob(os.path.join(folder, "*.mhd")):
            #print(filepath)
            img = sitk.ReadImage(filepath)
            output_path = os.path.join(outfolder, Path(filepath).stem + ".nii.gz")
            print(output_path)
            sitk.WriteImage(img, output_path)
#convert_LUNA16(basedir, outbasedir)

In [ ]:
basedir="/red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/"
outbasedir = "/red/ruogu.fang/yyang/data/TCIA_CT_Colon"
if not os.path.exists(outbasedir): os.makedirs(outbasedir)
folders = glob.glob(os.path.join(basedir,"*"))
patient_ids = [os.path.basename(f) for f in folders]

output_dir = os.path.join(outbasedir, 'images')

meta_data = []
count = 1
for i, patient_dir in enumerate(folders):
    #print(patient_dir) # to patient ID level
    for date_scan_dir in glob.glob(os.path.join(patient_dir,"*")): # to dates level 01-01-2000-1-Abdomen1ACRINCOLON Adult-16397
        #print(date_scan_dir)
        for scan_types_dir in glob.glob(os.path.join(date_scan_dir,"*")):
            #print(scan_types_dir)
            slices = len(os.listdir(scan_types_dir))
            
            if slices > 100:
               # try: 
                    filename = 'img_' + str(count) + '.nii.gz'
                    
                    dicom2nifti.convert_directory(scan_types_dir, output_dir) # This crate nii.gz name:  SeriesNumber_SeriesDescription.nii.gz or SeriesNumber_SequenceName.nii.gz, or other
                    
                    file_type = r'*nii.gz'
                    files = glob.glob(os.path.join(output_dir, file_type))
                    latest_file = max(files, key=os.path.getctime)
                    print(latest_file)
                    os.rename(Path(output_dir)/latest_file, Path(output_dir)/filename)

                    meta_entry = {'image':filename, 
                                  'slices': slices,
                                  'patient_id': os.path.basename(patient_dir),
                                  'date_scan_id': os.path.basename(date_scan_dir),
                                  'scan_type_id': os.path.basename(scan_types_dir)
                                 }
                    meta_data.append(meta_entry)
                    count += 1
                    print(f"{count}: {slices}, {scan_types_dir}")
                    
               # except: 
               #     print(f"Failed, skip {scan_types_dir}")
                    
                
json_object = json.dumps({'meta_data':meta_data}, indent = 4) 
with open(os.path.join(outbasedir, "CT_COLON_metadata_created.json"), "w") as outfile:
    json.dump(json_object, outfile)
    
print(f'Converted {len(meta_data)} images') 
#print(meta_data)

/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/2_colon_sup__10__b30f.nii.gz
2: 579, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0460/01-01-2000-1-Abdomen1ACRINCOLON Adult-16397/2.000000-COLON SUP  1.0  B30f-16403
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/5_prone_no_mirror_10__b30f.nii.gz
3: 606, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0460/01-01-2000-1-Abdomen1ACRINCOLON Adult-16397/5.000000-PRONE NO MIRROR 1.0  B30f-16983
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/3_recon_2_ct_colonography.nii.gz
4: 517, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0442/01-01-2000-1-CT COLONOSCOPY-95944/3.000000-Recon 2 CT COLONOGRAPHY-95948
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/11_recon_2_prone.nii.gz
5: 533, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAP

Removing duplicate slice from series


/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/4_prone_colon__st_10__b30f.nii.gz
7: 546, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0157/01-01-2000-1-Abdomen14SUPINECOLON Adult-76980/4.000000-Prone Colon  ST 1.0  B30f-76988
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/3_supine_colon__st_10__b30f.nii.gz
8: 561, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0157/01-01-2000-1-Abdomen14SUPINECOLON Adult-76980/3.000000-Supine Colon  ST 1.0  B30f-76993
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/5_colonography.nii.gz
9: 605, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0542/01-01-2000-1-CT COLONOGRAPHY SCREEN-10795/5.000000-COLONOGRAPHY-11364
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/3_colonography.nii.gz
10: 616, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRA

Slice increment not consistent through all slices
---------------------------------------------------------
[-199.60938 -370.60938 -428.7    ] [ 0.   0.  -0.8]
[-199.60938 -370.60938 -427.1    ] [ 0.   0.  -1.6]
Instance Number: 163
---------------------------------------------------------
Traceback (most recent call last):
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dicom.py", line 117, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_generic.py", line 132, in dicom_to_nifti
    common.validate_slice_increment(dicom_input)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/si

/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/img_108.nii.gz
110: 537, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0738/01-01-2000-17-Abdomen1 ACRIN PRONE COLONOGRAPHY-39934/4.000000-SUPINE 2MM AXIALS-40513
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/7_recon_2_prone.nii.gz
111: 535, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0627/01-01-2000-1-CT COLONOSCOPY-13557/7.000000-Recon 2 PRONE-14033
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/4_recon_2_ct_colonography.nii.gz
112: 468, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0627/01-01-2000-1-CT COLONOSCOPY-13557/4.000000-Recon 2 CT COLONOGRAPHY-13561
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/5_prone_colon__st_10__b30f.nii.gz
113: 536, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50

...

/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/img_992.nii.gz
994: 598, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0782/01-01-2000-57-Abdomen14SUPINECOLON Adult-90463/4.000000-Prone Colon  ST 1.0  B30f-91027
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/3_supine_colon__st_10__b30f.nii.gz
995: 553, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0782/01-01-2000-57-Abdomen14SUPINECOLON Adult-90463/3.000000-Supine Colon  ST 1.0  B30f-90473
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/4_colonography.nii.gz
996: 523, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0566/01-01-2000-1-CT COLONOGRAPHY SCREEN-39419/4.000000-COLONOGRAPHY-39971
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/2_colonography.nii.gz
997: 542, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.

Slice increment not consistent through all slices
---------------------------------------------------------
[-179.609375 -354.609375 -120.      ] [ 0.   0.  -0.8]
[-179.609375 -354.609375 -118.4     ] [ 0.   0.  -1.6]
Instance Number: 119
---------------------------------------------------------
Traceback (most recent call last):
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dicom.py", line 117, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_generic.py", line 132, in dicom_to_nifti
    common.validate_slice_increment(dicom_input)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python

/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/img_1025.nii.gz
1027: 659, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0727/01-01-2000-15-Abdomen24ACRINColoIRB2415-04 Adult-26322/3.000000-Colosupine  1.0  B30f-26329
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/2_supine.nii.gz
1028: 515, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0355/01-01-2000-1-CT COLONOGRAPHY SCREE-95948/2.000000-SUPINE-95952
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/4_prone.nii.gz
1029: 563, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0355/01-01-2000-1-CT COLONOGRAPHY SCREE-95948/4.000000-PRONE-96471
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/6_recon_2_abd_pel_without_-_pron.nii.gz
1030: 541, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0503/01-01-2000-1

Slice increment not consistent through all slices
---------------------------------------------------------
[-209.424961   -69.6631088  327.600037 ] [ 0.        0.       -0.800049]
[-209.424961   -69.6631088  329.200012 ] [ 0.        0.       -1.599975]
Instance Number: 411
---------------------------------------------------------
Traceback (most recent call last):
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dicom.py", line 117, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_generic.py", line 132, in dicom_to_nifti
    common.validate_slice_increment(dicom_input)
  File "/red/ufhpc/hity

/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/img_1039.nii.gz
1041: 538, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0449/01-01-2000-1-CT DIAGN VIRT COLONOSCOPY-03732/6.000000-SUPINE-03738
/red/ruogu.fang/yyang/data/TCIA_CT_Colon/images/10_prone.nii.gz
1042: 563, /red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0449/01-01-2000-1-CT DIAGN VIRT COLONOSCOPY-03732/10.000000-PRONE-04280


In [98]:
folder = "/red/ruogu.fang/cox.j/Downloads/manifest-sFI3R7DS3069120899390652954/CT COLONOGRAPHY/1.3.6.1.4.1.9328.50.4.0460/01-01-2000-1-Abdomen1ACRINCOLON Adult-16397/2.000000-COLON SUP  1.0  B30f-16403"

In [101]:
dicom2nifti.convert_directory(scan_types_dir, os.path.join(outbasedir, 'images'))

Removing duplicate slice from series
Traceback (most recent call last):
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_dicom.py", line 117, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/convert_generic.py", line 103, in dicom_to_nifti
    if common.is_multiframe_dicom(dicom_input):
  File "/red/ufhpc/hityangsir/.conda/envs/sitkpy/lib/python3.8/site-packages/dicom2nifti/common.py", line 148, in is_multiframe_dicom
    header = dicom_input[0]
IndexError: list index out of range
